In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell
from tensorflow.contrib import layers

### Parameters

In [2]:
# Parameters
learning_rate = 0.01
training_epochs = 1500
batch_size = 100
display_step = 100

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_input = 7 # Number of elements in input binary sequence 
n_classes = 2 # Total classes (even/odd) (0-1 digits)

# tf Graph input
x = tf.placeholder(tf.float32, [n_input])
y = tf.placeholder(tf.int32, [1])

# Define weights
weights_1 = {
    'out': tf.Variable(tf.random_normal([1]))
}
biases_1 = {
    'out': tf.Variable(tf.random_normal([1]))
}



### Data

In [3]:
## Train Sequence
train_X = np.random.randint(0, 2, size=(n_input))
train_Y = train_X.sum(keepdims=True)

In [4]:
## Test Sequence
test_X = np.random.randint(0, 2, size=(n_input))
test_Y = test_X.sum(keepdims=True)


### Building the Network

In [5]:
def RNN(x, n_hidden, weights, biases):

    x = [tf.expand_dims(x, 0)]
    # print(x.get_shape())

    # Define a lstm cell with tensorflow
    lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.mul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, n_hidden_1, weights_1, biases_1)[-1,-1]

# Define loss and optimizer
cost = tf.reduce_mean(pred-tf.cast(y, tf.float32))**2
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = pred
accuracy = cost

# Initializing the variables
init = tf.initialize_all_variables()

In [27]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    train_X = np.array([0,0,1,1,0,1,0])
    train_Y = train_X.sum(keepdims=True)    
    test_X = np.array([0,0,1,0,0,1,1])
    test_Y = test_X.sum(keepdims=True)
    print("True Train Sum: ", str(train_Y))
    for epoch in range(training_epochs):
        avg_cost = 0.
        # Loop over all batches
        # Run optimization op (backprop) and cost op (to get loss value)
        _, c = sess.run([optimizer, cost], feed_dict={x: train_X, y: train_Y})
        # Compute average loss
        avg_cost += c
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost) + ", sum = ", sess.run(pred, feed_dict={x: train_X}))
    print("Optimization Finished!")
    
    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Test Sum:", sess.run(pred, feed_dict={x: test_X}), ", True Sum:", str(test_Y))

True Train Sum:  [3]
Epoch: 0001 cost= 11.173274994, sum =  -0.323506
Epoch: 0101 cost= 1.155758858, sum =  1.93823
Epoch: 0201 cost= 0.062165115, sum =  2.75491
Epoch: 0301 cost= 0.001146707, sum =  2.9669
Epoch: 0401 cost= 0.000006311, sum =  2.99756
Epoch: 0501 cost= 0.000000008, sum =  2.99991
Epoch: 0601 cost= 0.000000000, sum =  3.0
Epoch: 0701 cost= 0.000000000, sum =  3.0
Epoch: 0801 cost= 0.000000000, sum =  3.0
Epoch: 0901 cost= 0.000000000, sum =  3.0
Epoch: 1001 cost= 0.000000000, sum =  3.0
Epoch: 1101 cost= 0.000000000, sum =  3.0
Epoch: 1201 cost= 0.000000000, sum =  3.0
Epoch: 1301 cost= 0.000000000, sum =  3.0
Epoch: 1401 cost= 0.000000000, sum =  3.0
Optimization Finished!
Test Sum: 2.88075 , True Sum: [3]
